In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, Input
from skimage.transform import resize
from skimage.io import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- CONFIGURACIÓN ---
DIRECTORIO_DATASET = os.path.abspath("./dataset")
ANCHO = 64
ALTO = 64
CANALES = 3

print(f"Buscando en: {DIRECTORIO_DATASET}")

if not os.path.exists(DIRECTORIO_DATASET):
    print("ERROR: No encuentro la carpeta 'dataset'.")
else:
    # 1. DETECTAR CLASES
    clases_validas = [
        d for d in os.listdir(DIRECTORIO_DATASET)
        if os.path.isdir(os.path.join(DIRECTORIO_DATASET, d))
    ]
    clases_validas.sort()

    mapa_clases = {nombre: i for i, nombre in enumerate(clases_validas)}

    print(f"Clases detectadas ({len(clases_validas)}): {clases_validas}")

    images = []
    labels = []

    # 2. CARGA DE IMÁGENES
    print("Cargando imágenes (esto puede tardar)...")

    for nombre_clase in clases_validas:
        ruta_clase = os.path.join(DIRECTORIO_DATASET, nombre_clase)
        archivos = os.listdir(ruta_clase)
        count = 0

        for archivo in archivos:
            if archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')):
                ruta_imagen = os.path.join(ruta_clase, archivo)

                try:
                    img = imread(ruta_imagen)

                    if len(img.shape) == 3 and img.shape[2] == 4:
                        img = img[:, :, :3]

                    img_resized = resize(
                        img, (ALTO, ANCHO),
                        anti_aliasing=True,
                        preserve_range=True
                    )

                    if img_resized.shape[-1] == 3:
                        images.append(img_resized)
                        labels.append(mapa_clases[nombre_clase])
                        count += 1

                except:
                    pass

        print(f"  {nombre_clase}: {count} imágenes")

    # 3. PROCESAMIENTO
    print("Procesando datos...")
    X = np.array(images, dtype=np.float32) / 255.0
    y = np.array(labels)

    y_one_hot = to_categorical(y, num_classes=len(clases_validas))

    print(f"Datos listos: X={X.shape}, y={y_one_hot.shape}")

    train_X, test_X, train_Y, test_Y = train_test_split(
        X, y_one_hot, test_size=0.2, random_state=42
    )

    # 4. MODELO CNN
    model = Sequential()
    model.add(Input(shape=(ALTO, ANCHO, CANALES)))

    # 🔹 BLOQUE 1
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(LeakyReLU(negative_slope=0.1))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    # 🔹 BLOQUE 2
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(LeakyReLU(negative_slope=0.1))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    # 🔥 BLOQUE 3
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(LeakyReLU(negative_slope=0.1))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    # Clasificación
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(negative_slope=0.1))
    model.add(Dropout(0.5))

    model.add(Dense(len(clases_validas), activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    # 5. DATA AUGMENTATION Y ENTRENAMIENTO
    print("Configurando Data Augmentation...")

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    print("Entrenando modelo con imágenes dinámicas...")

    history = model.fit(
        datagen.flow(train_X, train_Y, batch_size=32),
        epochs=25,
        validation_data=(test_X, test_Y)
    )

    # 6. GUARDAR MODELO
    model.save("modelo_animales.h5")
    np.save("clases.npy", clases_validas)

    print("¡Éxito Total! Modelo guardado como 'modelo_animales.h5'")


Buscando en: c:\Users\victo\Desktop\Tec\Semestre9\IA\dataset
Clases detectadas (5): ['ant', 'cat', 'dog', 'ladybug', 'turtle']
Cargando imágenes (esto puede tardar)...


c:\Users\victo\Desktop\Tec\Semestre9\IA\venv\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\victo\Desktop\Tec\Semestre9\IA\venv\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\victo\Desktop\Tec\Semestre9\IA\venv\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  ant: 7633 imágenes
  cat: 8000 imágenes
  dog: 8000 imágenes
  ladybug: 8000 imágenes
  turtle: 8000 imágenes
Procesando datos...
Datos listos: X=(39633, 64, 64, 3), y=(39633, 5)
Configurando Data Augmentation...
Entrenando modelo con imágenes dinámicas...
Epoch 1/25
991/991 ━━━━━━━━━━━━━━━━━━━━ 124s 123ms/step - accuracy: 0.6645 - loss: 0.7997 - val_accuracy: 0.6825 - val_loss: 0.7944
Epoch 2/25
991/991 ━━━━━━━━━━━━━━━━━━━━ 104s 105ms/step - accuracy: 0.7472 - loss: 0.6186 - val_accuracy: 0.7534 - val_loss: 0.6332
Epoch 3/25
991/991 ━━━━━━━━━━━━━━━━━━━━ 103s 104ms/step - accuracy: 0.7732 - loss: 0.5620 - val_accuracy: 0.7565 - val_loss: 0.5991
Epoch 4/25
991/991 ━━━━━━━━━━━━━━━━━━━━ 102s 103ms/step - accuracy: 0.7858 - loss: 0.5260 - val_accuracy: 0.7979 - val_loss: 0.5186
Epoch 5/25
991/991 ━━━━━━━━━━━━━━━━━━━━ 104s 105ms/step - accuracy: 0.7988 - loss: 0.4965 - val_accuracy: 0.7919 - val_loss: 0.5728
Epoch 6/25
991/991 ━━━━━━━━━━━━━━━━━━━━ 102s 103ms/step - accuracy: 0.8035 - loss

¡Éxito Total! Modelo guardado como 'modelo_animales.h5'
